In [14]:
import torch
import cv2
import glob
import os.path as osp
from torchvision import models
import torch.nn as nn
import datetime
import os

In [15]:
# 学習済みのVGG-16モデルをロード
# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットをアリとハチの2つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=4)

# 訓練モードに設定

print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')


/home/waki-lab/anaconda3/envs/wakilab/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/waki-lab/anaconda3/envs/wakilab/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


In [16]:
# PyTorchのネットワークパラメータのロード
load_path = '/home/waki-lab/デスクトップ/chiba/学習済み/weights_fine_tuning.pth'
#load_weights = torch.load(load_path)
#net.load_state_dict(load_weights)
# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)


<All keys matched successfully>

In [17]:
#モデルによって予測ラベルが生成されるかチェック
#画像読み込み
img=cv2.imread('/home/waki-lab/デスクトップ/chiba/画像/bunnkatu/001,001.jpg')
#tensorに変換(型を32ビット浮動小数点数に指定)
img=torch.tensor(img,dtype=torch.float32)
print(img.size())
#0番目にバッチサイズ分の次元を追加
img=img.unsqueeze(0)
print(img.size())
#(バッチサイズ,高さ,幅,色)を(バッチサイズ,色,幅,高さ)に変換
img=torch.permute(img,(0,3,2,1))
print(img.size())
net.to('cuda:0')
net.eval() 
img = img.to('cuda:0')
result=net(img)
_, preds = torch.max(result, 1)
#予測ラベル
print(preds)
print(preds.item())

torch.Size([172, 154, 3])
torch.Size([1, 172, 154, 3])
torch.Size([1, 3, 154, 172])
tensor([3], device='cuda:0')
3


In [18]:
#ファイルのパスのリストを制作
def make_datapath_list2():

    rootpath = "/home/waki-lab/デスクトップ/chiba/画像/bunnkatu"
    target_path = osp.join(rootpath+'/*.jpg')
    print(target_path)

    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list
mylist=make_datapath_list2()

/home/waki-lab/デスクトップ/chiba/画像/bunnkatu/*.jpg


In [19]:
#フォルダ制作
now=str(datetime.datetime.now())
new_dir_path='/home/waki-lab/デスクトップ/chiba/グループ分け/'+now
os.mkdir(new_dir_path)
mori_dir_path=new_dir_path+'/森'
os.mkdir(mori_dir_path)
umi_dir_path=new_dir_path+'/海'
os.mkdir(umi_dir_path)
suna_dir_path=new_dir_path+'/砂'
os.mkdir(suna_dir_path)
siro_dir_path=new_dir_path+'/白'
os.mkdir(siro_dir_path)

In [20]:
#let'sグループ分け
for path in mylist:
    #画像読み込み
    IMG=cv2.imread(path)
    #tensorに変換(型を32ビット浮動小数点数に指定)
    img=torch.tensor(IMG,dtype=torch.float32)
    #print(img.size())
    #0番目にバッチサイズ分の次元を追加
    img=img.unsqueeze(0)
    #print(img.size())
    #(バッチサイズ,高さ,幅,色)を(バッチサイズ,色,幅,高さ)に変換
    img=torch.permute(img,(0,3,2,1))
    #print(img.size())
    net.to('cuda:0')
    net.eval() 
    img = img.to('cuda:0')
    result=net(img)
    _, preds = torch.max(result, 1)
    #予測ラベル
    pred=str(preds.item())
    #ラベルを名前に変換
    if   pred==str(0):
          pred='森'
    elif pred==str(1):
          pred='砂'
    elif pred==str(2):
          pred='海'
    elif pred==str(3):
          pred='白'
    #予測ラベルをもとに画像を保存
    namepath=path[40:52]
    cv2.imwrite('/home/waki-lab/デスクトップ/chiba/グループ分け/'+now+'/'+pred+"/"+namepath, IMG)